In [1]:
class SomeObject:
    def __init__(self):
        self.integer_field = 0
        self.float_field = 0.0
        self.string_field = ""






In [49]:
class NullHandler:
    def __init__(self, successor=None):
        self.__successor = successor
    
    def handle(self, obj, event):
        if self.__successor is not None:
            return self.__successor.handle(obj, event)
        else:
            return None
    

            
class FloatHandler(NullHandler): 
    def handle(self, obj, event):
        if event.dtype == float:
            #print(f"{event.kind} {event.dtype} value in FloatHandler")
            if event.kind == 'get':
                print(f"returning {obj.float_field} ")
                return obj.float_field
            elif event.kind == 'set':
                obj.float_field = event.data
                return None
        else:
            #print("Передаю обработку дальше")
            return super().handle(obj, event)
            
class IntHandler(NullHandler):
    def handle(self, obj, event):
        if event.dtype == int:
            #print(f"{event.kind} {event.dtype} value in IntHandler")
            if event.kind == 'get':
                return obj.integer_field
            elif event.kind == 'set':
                obj.integer_field = event.data
                return None
        else:
            #print("Передаю обработку дальше")
            return super().handle(obj, event)

class StrHandler(NullHandler): 
    def handle(self, obj, event):
        if event.dtype == str:
            #print(f"{event.kind} {event.dtype} value in StrHandler")
            if event.kind == 'get':
                return obj.str_field
            elif event.kind == 'set':
                obj.str_field = event.data
                return None
        else:
            #print("Передаю обработку дальше")
            return super().handle(obj, event)


In [50]:
class Event:
    def __init__(self, t):
        self.kind = None
        self.dtype = None
        
class EventGet(Event):
    def __init__(self, t):
        self.kind = "get"
        self.dtype = t
        self.data = None

class EventSet(Event):
    def __init__(self, data):
        self.dtype = type(data)
        self.kind = "set"
        self.data = data

In [51]:
obj = SomeObject()
obj.integer_field = 2
obj.float_field = 103.2
obj.str_field = 'dummy'

chain = IntHandler(FloatHandler(StrHandler(NullHandler())))



In [52]:
print(chain.handle(obj, EventGet(float)))

Передаю обработку дальше
get <class 'float'> value in FloatHandler
returning 103.2 
103.2


In [53]:
print(chain.handle(obj, EventGet(str)))

Передаю обработку дальше
Передаю обработку дальше
get <class 'str'> value in StrHandler
dummy


In [54]:
print(chain.handle(obj, EventGet(int)))

get <class 'int'> value in IntHandler
2


In [57]:
chain.handle(obj, EventSet(117))
chain.handle(obj, EventGet(int))

set <class 'int'> value in IntHandler
get <class 'int'> value in IntHandler


117

In [59]:
chain.handle(obj, EventSet(117.117))
chain.handle(obj, EventGet(float))

Передаю обработку дальше
set <class 'float'> value in FloatHandler
Передаю обработку дальше
get <class 'float'> value in FloatHandler
returning 117.117 


117.117

In [60]:
chain.handle(obj, EventSet("blabla"))
chain.handle(obj, EventGet(str))

Передаю обработку дальше
Передаю обработку дальше
set <class 'str'> value in StrHandler
Передаю обработку дальше
Передаю обработку дальше
get <class 'str'> value in StrHandler


'blabla'